# Module 3. Invoke SageMaker Endpoint
---

SageMaker 엔드포인트가 구성되었다고 가정합니다. 아래 코드는 참고용으로만 확인하세요.

In [47]:
PROJECT = "amzn-reviews"
!rm -rf $PROJECT
!chalice new-project $PROJECT

Your project has been generated in ./amzn-reviews


In [48]:
cat $PROJECT/.chalice/config.json

{
  "version": "2.0",
  "app_name": "amzn-reviews",
  "stages": {
    "dev": {
      "api_gateway_stage": "api"
    }
  }
}


In [49]:
!tree -a $PROJECT/.

amzn-reviews/.
├── app.py
├── .chalice
│   └── config.json
├── .gitignore
└── requirements.txt

1 directory, 4 files


In [50]:
%store -r
%store

Stored variables and their in-db values:
challenger_model_artifact                    -> 's3://sagemaker-us-east-1-143656149352/mab-reviews
champion_model_artifact                      -> 's3://sagemaker-us-east-1-143656149352/mab-reviews
endpoint_name                                -> 'MAP-Inference-Endpoint-2023-03-14-02-56-51'
model_train_inference_docker_uri             -> '811284229777.dkr.ecr.us-east-1.amazonaws.com/blaz


In [51]:
endpoint_name

'MAP-Inference-Endpoint-2023-03-14-02-56-51'

In [52]:
cat $PROJECT/.chalice/config.json

{
  "version": "2.0",
  "app_name": "amzn-reviews",
  "stages": {
    "dev": {
      "api_gateway_stage": "api"
    }
  }
}


#### Setup config.json
Chalice는 IAM 정책 자동 생성 기능이 있지만, 필요한 정책을 가진 IAM 정책을 생성할수 있습니다. 기본적으로는 직접 IAM 정책을 생성하는 것이 안전합니다. <br>
자세한 내용은 https://chalice-fei.readthedocs.io/en/latest/topics/configfile.html 를 참조하기 바랍니다.

`autogen_policy`: 
- 애플리케이션 소스 코드 분석을 기반으로 chalice가 IAM 정책을 자동으로 생성할지 여부를 설정 (디폴트 = True)
- False인 경우, `.chalice/policy-<단계 이름>.json`에서 IAM 정책을 로드
- `iam_policy_file` 지정으로 불러올 policy 파일명을 변경할 수도 있음

In [56]:
%%writefile $PROJECT/.chalice/config.json

{
    "Version": "2.0",
    "app_name": "amzn-reviews",
    "autogen_policy": false,
    "automatic_layer": true,
    "environment_variables": {
        "ENDPOINT_NAME": "MAP-Inference-Endpoint-2023-03-14-02-56-51"
    },
    "stages": {
        "dev": {
            "api_gateway_stage": "api"
        }    
    }

}

Overwriting amzn-reviews/.chalice/config.json


아래와 같이 Private VPC 환경을 구축할 수도 있습니다.

In [54]:
# {
#     "Version": "2.0",
#     "app_name": "amzn-reviews",
#     "autogen_policy": false,
#     "automatic_layer": true,
#     "environment_variables": {
#         "ENDPOINT_NAME": "MAP-Inference-Endpoint-2023-03-14-02-56-51"
#     },
#     "stages": {
#         "dev": {
#             "api_gateway_stage": "api"
#             "iam_policy_file": "policy-dev-1.json"
#             "subnet_ids": ["subnet-xxxxxxxxxxxxxx", "subnet-xxxxxxxxxxxxxx"],
#             "security_group_ids": ["sg-xxxxxxxxxxxxxx"],
#         }
#     }

# }

#### Setup IAM policy

In [55]:
%%writefile $PROJECT/.chalice/policy-dev.json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents",
                "logs:CreateLogGroup"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        },
        {
            "Sid": "VisualEditor1",
            "Effect": "Allow",
            "Action": "sagemaker:InvokeEndpoint",
            "Resource": "*"
        }
    ]
}

Writing amzn-reviews/.chalice/policy-dev.json


#### app.py

In [57]:
%%writefile $PROJECT/app.py 
import os
import io
import json
import boto3
import base64
import numpy as np

from chalice import Chalice
from chalice import BadRequestError

app = Chalice(app_name="amzn-reviews")
app.debug = True

runtime = boto3.client("runtime.sagemaker")

@app.route("/")
def index():
    return {'hello': 'world'}

@app.route("/variant/{variant_name}", methods=["POST"])
def return_val(variant_name):
    body = app.current_request.json_body
    if "data" not in body:
        raise BadRequestError("Missing data.")
    if "ENDPOINT_NAME" not in os.environ:
        raise BadRequestError("Missing endpoint.")

    endpoint_name = os.environ["ENDPOINT_NAME"]

    array = body['data']
    payload = {"instances" : array, "configuration": {"k": 1} }

    print(array)

    def parse_predictions(results):
        return [(result['label'][0] == '__label__Helpful', result['prob'][0]) for result in results]    

    try:
        response = runtime.invoke_endpoint(
            EndpointName = endpoint_name, 
            TargetVariant = variant_name,
            ContentType = 'application/json',                        
            Body = json.dumps(payload))
        predictions = json.loads(response['Body'].read())            
        print(parse_predictions(predictions))
        
        return {
            "response": {
                "value": json.dumps(parse_predictions(predictions))
            }
        }
    except Exception as e:
        print(e)
        print(payload)
    


Overwriting amzn-reviews/app.py


In [58]:
%%writefile $PROJECT/requirements.txt
numpy

Overwriting amzn-reviews/requirements.txt


### Local Test
로컬 환경에서 편리하게 테스트를 수행할 수 있습니다.


In [59]:
!cd $PROJECT && chalice local --port=8100

Restarting local dev server.
Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
Serving on http://127.0.0.1:8100
['Very bad product. I hate it']
[(False, 0.9934085607528687)]
127.0.0.1 - - [14/Mar/2023 07:52:27] "POST /variant/Champion HTTP/1.1" 200 -
^C


```

curl -X POST localhost:8100/variant/Champion -H "Content-Type: application/json" -d '{"data":["Very bad product. I hate it"]}'

curl -X POST localhost:8100/variant/Challenger -H "Content-Type: application/json" -d '{"data":["Very bad product. I hate it"]}'

```

### Deploy

In [60]:
!cd $PROJECT && chalice deploy

Creating shared layer deployment package.
Creating app deployment package.
Creating lambda layer: amzn-reviews-dev-managed-layer
Creating IAM role: amzn-reviews-dev-api_handler
Creating lambda function: amzn-reviews-dev
Creating Rest API
Resources deployed:
  - Lambda Layer ARN: arn:aws:lambda:us-east-1:143656149352:layer:amzn-reviews-dev-managed-layer:9
  - Lambda ARN: arn:aws:lambda:us-east-1:143656149352:function:amzn-reviews-dev
  - Rest API URL: https://2mvnqeabm7.execute-api.us-east-1.amazonaws.com/api/


In [61]:
from IPython.core.display import display, HTML
import boto3
client = boto3.client('apigateway')
region = boto3.Session().region_name
response = client.get_rest_apis(limit=2)
restapi_id = response['items'][0]['id']
url = f'https://{restapi_id}.execute-api.{region}.amazonaws.com/api/'.replace('"','')

In [62]:
!curl -X POST https://2mvnqeabm7.execute-api.us-east-1.amazonaws.com/api/variant/Champion \
  -H "Content-Type: application/json" \
  -d '{"data":["Very bad product. I hate it"]}'

{"response":{"value":"[[false, 0.9934085607528687]]"}}

In [63]:
!curl -X POST https://2mvnqeabm7.execute-api.us-east-1.amazonaws.com/api/variant/Challenger \
  -H "Content-Type: application/json" \
  -d '{"data":["Very bad product. I hate it"]}'

{"response":{"value":"[[false, 0.9977796077728271]]"}}

In [66]:
!cd $PROJECT && chalice delete